## Chatbot interactiv Phi 3 Mini 4K Instruct cu Whisper

### Introducere:
Chatbotul interactiv Phi 3 Mini 4K Instruct este un instrument care permite utilizatorilor să interacționeze cu demonstrația Microsoft Phi 3 Mini 4K Instruct folosind intrări text sau audio. Chatbotul poate fi utilizat pentru o varietate de sarcini, cum ar fi traduceri, actualizări meteo și colectarea de informații generale.


In [ ]:
#Install required Python Packages
!pip install accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True
!pip install transformers
!pip install wheel
!pip install gradio
!pip install pydub==0.25.1
!pip install edge-tts
!pip install openai-whisper==20231117
!pip install ffmpeg==1.4
# from IPython.display import clear_output
# clear_output()

In [ ]:
# Checking to see if Cuda support is available 
# Output True = Cuda
# Output False = No Cuda (installing Cuda will be required to run the model on GPU)
import os 
import torch
print(torch.cuda.is_available())


[Creează-ți Token-ul de Acces Huggingface](https://huggingface.co/settings/tokens)

Creează un token nou  
Furnizează un nume nou  
Selectează permisiunile de scriere  
Copiază token-ul și salvează-l într-un loc sigur  


Următorul cod Python îndeplinește două sarcini principale: importarea modulului `os` și setarea unei variabile de mediu.

1. Importarea modulului `os`:
   - Modulul `os` din Python oferă o modalitate de a interacționa cu sistemul de operare. Permite efectuarea diverselor sarcini legate de sistemul de operare, cum ar fi accesarea variabilelor de mediu, lucrul cu fișiere și directoare etc.
   - În acest cod, modulul `os` este importat folosind instrucțiunea `import`. Această instrucțiune face ca funcționalitatea modulului `os` să fie disponibilă pentru utilizare în scriptul Python curent.

2. Setarea unei variabile de mediu:
   - O variabilă de mediu este o valoare care poate fi accesată de programele care rulează pe sistemul de operare. Este o modalitate de a stoca setări de configurare sau alte informații care pot fi utilizate de mai multe programe.
   - În acest cod, o nouă variabilă de mediu este setată folosind dicționarul `os.environ`. Cheia dicționarului este `'HF_TOKEN'`, iar valoarea este atribuită din variabila `HUGGINGFACE_TOKEN`.
   - Variabila `HUGGINGFACE_TOKEN` este definită chiar deasupra acestui fragment de cod și i se atribuie o valoare de tip șir de caractere `"hf_**************"` folosind sintaxa `#@param`. Această sintaxă este adesea utilizată în Jupyter notebooks pentru a permite introducerea de date de către utilizator și configurarea parametrilor direct în interfața notebook-ului.
   - Prin setarea variabilei de mediu `'HF_TOKEN'`, aceasta poate fi accesată de alte părți ale programului sau de alte programe care rulează pe același sistem de operare.

În concluzie, acest cod importă modulul `os` și setează o variabilă de mediu numită `'HF_TOKEN'` cu valoarea furnizată în variabila `HUGGINGFACE_TOKEN`.


In [ ]:
import os
# set the Hugging Face Token from 
# add the Hugging Face Token to the environment variables
HUGGINGFACE_TOKEN = "Enter Hugging Face Key" #@param {type:"string"}
os.environ['HF_TOKEN']HUGGINGFACE_TOKEN

Acest fragment de cod definește o funcție numită clear_output, care este utilizată pentru a șterge rezultatul afișat în celula curentă din Jupyter Notebook sau IPython. Haideți să analizăm codul și să înțelegem funcționalitatea sa:

Funcția clear_output primește un parametru numit wait, care este o valoare booleană. În mod implicit, wait este setat la False. Acest parametru determină dacă funcția ar trebui să aștepte până când este disponibil un nou rezultat pentru a înlocui rezultatul existent înainte de a-l șterge.

Funcția în sine este utilizată pentru a șterge rezultatul afișat în celula curentă. În Jupyter Notebook sau IPython, atunci când o celulă produce un rezultat, cum ar fi text imprimat sau grafice, acel rezultat este afișat sub celulă. Funcția clear_output permite ștergerea acelui rezultat.

Implementarea funcției nu este furnizată în fragmentul de cod, așa cum este indicat de punctele de suspensie (...). Punctele de suspensie reprezintă un substitut pentru codul real care realizează ștergerea rezultatului. Implementarea funcției poate implica interacțiunea cu API-ul Jupyter Notebook sau IPython pentru a elimina rezultatul existent din celulă.

În concluzie, această funcție oferă o modalitate convenabilă de a șterge rezultatul afișat în celula curentă din Jupyter Notebook sau IPython, facilitând gestionarea și actualizarea rezultatelor afișate în timpul sesiunilor interactive de programare.


In [ ]:
# Download Phi-3-mini-4k-instruct model & Whisper Tiny
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#whisper for speech to text()
import whisper
select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)

#from IPython.display import clear_output
#clear_output()

Efectuați conversia text-în-vorbire (TTS) utilizând serviciul Edge TTS. Să analizăm implementările funcțiilor relevante una câte una:

1. `calculate_rate_string(input_value)`: Această funcție primește o valoare de intrare și calculează șirul de rată pentru vocea TTS. Valoarea de intrare reprezintă viteza dorită a vorbirii, unde o valoare de 1 reprezintă viteza normală. Funcția calculează șirul de rată scăzând 1 din valoarea de intrare, înmulțind rezultatul cu 100 și apoi determinând semnul în funcție de faptul dacă valoarea de intrare este mai mare sau egală cu 1. Funcția returnează șirul de rată în formatul "{semn}{rată}".

2. `make_chunks(input_text, language)`: Această funcție primește un text de intrare și o limbă ca parametri. Ea împarte textul de intrare în fragmente pe baza regulilor specifice limbii. În această implementare, dacă limba este "English", funcția împarte textul la fiecare punct (".") și elimină orice spațiu alb de la început sau sfârșit. Apoi adaugă un punct la fiecare fragment și returnează lista filtrată de fragmente.

3. `tts_file_name(text)`: Această funcție generează un nume de fișier pentru fișierul audio TTS pe baza textului de intrare. Ea efectuează mai multe transformări asupra textului: elimină un punct final (dacă este prezent), convertește textul în litere mici, elimină spațiile albe de la început și sfârșit și înlocuiește spațiile cu liniuțe de subliniere. Apoi, textul este trunchiat la maximum 25 de caractere (dacă este mai lung) sau se folosește textul complet dacă este gol. În final, generează un șir aleatoriu utilizând modulul [`uuid`] și îl combină cu textul trunchiat pentru a crea numele fișierului în formatul "/content/edge_tts_voice/{truncated_text}_{random_string}.mp3".

4. `merge_audio_files(audio_paths, output_path)`: Această funcție îmbină mai multe fișiere audio într-un singur fișier audio. Primește o listă de căi ale fișierelor audio și o cale de ieșire ca parametri. Funcția inițializează un obiect gol `AudioSegment` numit [`merged_audio`]. Apoi, parcurge fiecare cale a fișierului audio, încarcă fișierul audio utilizând metoda `AudioSegment.from_file()` din biblioteca `pydub` și adaugă fișierul audio curent la obiectul [`merged_audio`]. În final, exportă audio-ul îmbinat la calea de ieșire specificată în formatul MP3.

5. `edge_free_tts(chunks_list, speed, voice_name, save_path)`: Această funcție efectuează operațiunea TTS utilizând serviciul Edge TTS. Primește o listă de fragmente text, viteza vorbirii, numele vocii și calea de salvare ca parametri. Dacă numărul de fragmente este mai mare de 1, funcția creează un director pentru stocarea fișierelor audio individuale ale fragmentelor. Apoi, parcurge fiecare fragment, construiește o comandă Edge TTS utilizând funcția `calculate_rate_string()`, numele vocii și textul fragmentului, și execută comanda utilizând funcția `os.system()`. Dacă execuția comenzii are succes, adaugă calea fișierului audio generat într-o listă. După procesarea tuturor fragmentelor, îmbină fișierele audio individuale utilizând funcția `merge_audio_files()` și salvează audio-ul îmbinat la calea de salvare specificată. Dacă există un singur fragment, generează direct comanda Edge TTS și salvează audio-ul la calea de salvare. În final, returnează calea de salvare a fișierului audio generat.

6. `random_audio_name_generate()`: Această funcție generează un nume aleatoriu pentru fișierul audio utilizând modulul [`uuid`]. Generează un UUID aleatoriu, îl convertește într-un șir, ia primele 8 caractere, adaugă extensia ".mp3" și returnează numele aleatoriu al fișierului audio.

7. `talk(input_text)`: Această funcție este punctul principal de intrare pentru efectuarea operațiunii TTS. Primește un text de intrare ca parametru. Mai întâi verifică lungimea textului de intrare pentru a determina dacă este o propoziție lungă (mai mare sau egală cu 600 de caractere). Pe baza lungimii și a valorii variabilei `translate_text_flag`, determină limba și generează lista de fragmente text utilizând funcția `make_chunks()`. Apoi generează o cale de salvare pentru fișierul audio utilizând funcția `random_audio_name_generate()`. În final, apelează funcția `edge_free_tts()` pentru a efectua operațiunea TTS și returnează calea de salvare a fișierului audio generat.

În ansamblu, aceste funcții lucrează împreună pentru a împărți textul de intrare în fragmente, a genera un nume de fișier pentru fișierul audio, a efectua operațiunea TTS utilizând serviciul Edge TTS și a îmbina fișierele audio individuale într-un singur fișier audio.


In [ ]:
#@title Edge TTS
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path

# text = "This is Microsoft Phi 3 mini 4k instruct Demo" Simply update the text variable with the text you want to convert to speech
text = 'This is Microsoft Phi 3 mini 4k instruct Demo'  # @param {type: "string"}
Language = "English" # @param ['English']
# Gender of voice simply change from male to female and choose the voice you want to use
Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
if len(text)>=600:
  long_sentence = True
else:
  long_sentence = False

# long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global translate_text_flag,Language,speed,voice_name
  if len(input_text)>=600:
    long_sentence = True
  else:
    long_sentence = False

  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

Implementarea a două funcții: convert_to_text și run_text_prompt, precum și declarația a două clase: str și Audio.

Funcția convert_to_text primește ca input un audio_path și transcrie audio-ul în text folosind un model numit whisper_model. Funcția verifică mai întâi dacă flag-ul gpu este setat pe True. Dacă este, whisper_model este utilizat cu anumiți parametri, cum ar fi word_timestamps=True, fp16=True, language='English' și task='translate'. Dacă flag-ul gpu este False, whisper_model este utilizat cu fp16=False. Transcrierea rezultată este apoi salvată într-un fișier numit 'scan.txt' și returnată ca text.

Funcția run_text_prompt primește un mesaj și un chat_history ca input. Aceasta folosește funcția phi_demo pentru a genera un răspuns de la un chatbot pe baza mesajului de intrare. Răspunsul generat este apoi transmis funcției talk, care convertește răspunsul într-un fișier audio și returnează calea fișierului. Clasa Audio este utilizată pentru a afișa și reda fișierul audio. Audio-ul este afișat folosind funcția display din modulul IPython.display, iar obiectul Audio este creat cu parametrul autoplay=True, astfel încât audio-ul începe să se redea automat. chat_history este actualizat cu mesajul de intrare și răspunsul generat, iar un șir gol și chat_history actualizat sunt returnate.

Clasa str este o clasă încorporată în Python care reprezintă o secvență de caractere. Aceasta oferă diverse metode pentru manipularea și lucrul cu șiruri de caractere, cum ar fi capitalize, casefold, center, count, encode, endswith, expandtabs, find, format, index, isalnum, isalpha, isascii, isdecimal, isdigit, isidentifier, islower, isnumeric, isprintable, isspace, istitle, isupper, join, ljust, lower, lstrip, partition, replace, removeprefix, removesuffix, rfind, rindex, rjust, rpartition, rsplit, rstrip, split, splitlines, startswith, strip, swapcase, title, translate, upper, zfill și altele. Aceste metode permit efectuarea de operații precum căutarea, înlocuirea, formatarea și manipularea șirurilor de caractere.

Clasa Audio este o clasă personalizată care reprezintă un obiect audio. Este utilizată pentru a crea un player audio în mediul Jupyter Notebook. Clasa acceptă diferiți parametri, cum ar fi data, filename, url, embed, rate, autoplay și normalize. Parametrul data poate fi un array numpy, o listă de mostre, un șir care reprezintă un nume de fișier sau URL, sau date PCM brute. Parametrul filename este utilizat pentru a specifica un fișier local din care să se încarce datele audio, iar parametrul url este utilizat pentru a specifica un URL de unde să se descarce datele audio. Parametrul embed determină dacă datele audio ar trebui să fie încorporate folosind un URI de date sau referite din sursa originală. Parametrul rate specifică rata de eșantionare a datelor audio. Parametrul autoplay determină dacă audio-ul ar trebui să înceapă să se redea automat. Parametrul normalize specifică dacă datele audio ar trebui să fie normalizate (rescalate) la intervalul maxim posibil. Clasa Audio oferă, de asemenea, metode precum reload pentru a reîncărca datele audio din fișier sau URL, și atribute precum src_attr, autoplay_attr și element_id_attr pentru a obține atributele corespunzătoare pentru elementul audio în HTML.

În general, aceste funcții și clase sunt utilizate pentru a transcrie audio în text, a genera răspunsuri audio de la un chatbot și a afișa și reda audio în mediul Jupyter Notebook.


In [ ]:
#@title Run gradio app
def convert_to_text(audio_path):
  gpu=True
  if gpu:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=True,language='English',task='translate')
  else:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=False,language='English',task='translate')
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  return result["text"]


import gradio as gr
from IPython.display import Audio, display
def run_text_prompt(message, chat_history):
    bot_message = phi_demo(message)
    edge_save_path=talk(bot_message)
    # print(edge_save_path)
    display(Audio(edge_save_path, autoplay=True))

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history
    print(audio)
    message_transcription = convert_to_text(audio)
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with Phi 3 mini 4k instruct")

    msg = gr.Textbox(label="Ask anything")
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(share=True,debug=True)


---

**Declinare de responsabilitate**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși ne străduim să asigurăm acuratețea, vă rugăm să fiți conștienți că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa natală ar trebui considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist uman. Nu ne asumăm responsabilitatea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea din utilizarea acestei traduceri.
